<a href="https://colab.research.google.com/github/withpi/cookbook-withpi/blob/main/colabs/Sagemaker_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://withpi.ai"><img src="https://withpi.ai/logo/logoFullBlack.svg" width="240px"></a>

<a href="https://code.withpi.ai"><font size="4">Documentation</font></a>

<a href="https://withpi.ai"><font size="4">Copilot</font></a>

# Embeddings

Pi has published an API for performing embedding inference.

It takes as input a list of items to embed and returns a list of embeddings.

It can be deployed to Sagemaker for inference in your own account.  This notebook shows how to perform inference with it.

In [ ]:
%pip install httpx tqdm

import os
from google.colab import userdata

os.environ["WITHPI_API_KEY"] = userdata.get('WITHPI_API_KEY')


In [ ]:
import asyncio
import httpx
import time

async with httpx.AsyncClient() as client:
  latencies = []
  for _ in range(10):
    start = time.perf_counter()
    resp = await client.post(
      "https://api.withpi.ai/v1/search/embed",
      headers={
        "x-api-key": os.environ['WITHPI_API_KEY'],
      },
      json={
        "query": ["Some document to embed"],
        "batch": False,
      },
      timeout=10.0
    )
    stop = time.perf_counter()
    latencies.append(f"{stop-start:.3f}")

  resp.raise_for_status()
  print(f"Tokens processed: {resp.headers['x-tokens-processed']}")
  print(f"Latencies: {latencies}")
  parsed = resp.json()
  print(f"Number of embeddings: {len(parsed)}")
  print(f"Dimensionality of embeddings: {len(parsed[0])}")
  print(f"Sample: {parsed[0][:5]}")

Tokens processed: 6
Latencies: ['0.736', '0.382', '0.143', '0.142', '0.136', '0.479', '0.137', '0.146', '0.140', '0.143']
Number of embeddings: 1
Dimensionality of embeddings: 256
Sample: [-0.08392333984375, -0.072998046875, -0.00856781005859375, -0.039306640625, 0.08184814453125]


## Calling at scale

Any number of documents can be sent in one request.  The backend maintains
a queue and will dynamically group small requests into larger batches for throughput.  It will also slice overly large requests into pieces for execution.

The "batch" parameter controls whether to hit an ONNX model tuned to a batch size of 1 (for minimal online latency) or with a Torch model configured to a batch size of 32 (to maximize throughput).  The input list can be any length, but picking one of those two sizes will maximize throughput.

We recommend limiting concurrency as a throttling mechanism, since that will avoid runaway queue growth.  See below for an example that sends 50k documents quickly.



In [ ]:
import asyncio
from itertools import islice
import httpx
import random
import time
import tqdm.asyncio

class Counter:
  def __init__(self):
    self.token_count = 0
  def increment(self, amount):
    self.token_count += amount

def make_fake_documents():
  docs = []
  for _ in range(50000):
    docs.append(" ".join(["word"]*random.randint(512, 512*3)))
  return docs

def batched(iterable, n):
    "Batch data into tuples of length n. The last batch may be shorter."
    # batched('ABCDEFG', 3) --> ABC DEF G
    if n < 1:
        raise ValueError('n must be at least one')
    it = iter(iterable)
    while batch := tuple(islice(it, n)):
        yield batch

corpus = make_fake_documents()
concurrency_limit = asyncio.Semaphore(64)
token_counter = Counter()

async with httpx.AsyncClient() as client:
  async def call_batch(batch):
    async with concurrency_limit:
      attempts = 0
      while attempts < 3:
        attempts += 1
        resp = await client.post(
          "https://api.withpi.ai/v1/search/embed",
          headers={
            "x-api-key": os.environ['WITHPI_API_KEY'],
          },
          json={
            "query": batch,
            "batch": True,
          },
          timeout=10.0
        )
        if resp.status_code == 200:
          # No lock because this is single-threaded asyncio.
          # Othewise protect this.
          token_counter.increment(int(resp.headers['x-tokens-processed']))
          return
        # Sleep while holding the concurrency limiter to force a slowdown.
        print(f"Status code: {resp.status_code}")
        print(f"Retrying after {attempts} seconds")
        await asyncio.sleep(attempts)
      raise ValueError("Retries exhausted...shutting down")

  start = time.perf_counter()
  try:
    async with asyncio.TaskGroup() as tg:
      tasks = []
      for batch in batched(corpus, 32):
        tasks.append(tg.create_task(call_batch(batch)))
      for f in tqdm.asyncio.tqdm.as_completed(tasks):
        await f
  except ExceptionGroup as e:
    print(e.exceptions)
    raise


  stop = time.perf_counter()
  elapsed = stop - start

  print("\n")
  print(f"Total tokens processed: {token_counter.token_count}")
  print(f"Elapsed time: {elapsed:.2f} seconds")
  print(f"Throughput: {token_counter.token_count / elapsed:.2f} tokens/second")


100%|██████████| 1563/1563 [00:55<00:00, 28.04it/s]



Total tokens processed: 51250480
Elapsed time: 55.78 seconds
Throughput: 918782.43 tokens/second
